In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sales_train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [ ]:
oct_trans=sales_train[sales_train["date"].str.endswith("10.2015")]
oct_sales = oct_trans.groupby(['shop_id','item_id'])['item_cnt_day'].sum()
oct_sales

In [ ]:
test

In [ ]:
merged=pd.merge(test,oct_sales,on=['shop_id','item_id'],how="outer")
#merged=merged[merged['ID']!=np.nan]
merged=merged.dropna(subset=['ID'])
merged=merged.replace(np.nan,0)
merged['item_cnt_day']=merged['item_cnt_day'].clip(lower=0,upper=20)
merged=merged.drop(['shop_id','item_id'])
merged